In [1]:
from warnings import simplefilter 
import os
simplefilter(action='ignore', category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3" 
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from kvae.KalmanVariationalAutoencoder import KalmanVariationalAutoencoder
from kvae.utils import reload_config, get_image_config

%matplotlib inline
plt.style.use("seaborn-white")

In [3]:
n_lat = 2

In [4]:
if n_lat == 2:
    logdir = Path("../examples/logs/2dspring_success_kvae/model.ckpt")
else:
    logdir = Path("../examples/logs/2dspring_success_kvae_16dim/model.ckpt")

In [5]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [6]:
config = get_image_config()
# To reload a saved model
config = reload_config(config.FLAGS)

In [7]:
config.reload_model = str(logdir)
config.batch_size = 32
config.dataset = "spring_kfixlfix"
config.K = 7
config.dim_a = n_lat

In [8]:
sess = tf.Session()
model = KalmanVariationalAutoencoder(config, sess)
model.build_model().build_loss().initialize_variables()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions f

In [9]:
mask_test_full = np.ones((model.config.batch_size, model.test_data.timesteps), dtype=np.float32)
mask_test_intermittent = 1 - mask_test_full.copy()
mask_test_intermittent[:, ::10] = 1
mask_test_intermittent[:, -1] = 1
mask_test_predict = mask_test_full.copy()
mask_test_predict[:, 60:] = 0
mask_test_impute = mask_test_full.copy()
mask_test_impute[:, 30:50] = 0
# mask_test[:, 20:60] = 0
num_batches = model.test_data.sequences // model.config.batch_size

for pattern in ["full", "predict", "impute", "intermittent"]:
    mask_test = eval("mask_test_{}".format(pattern))
    x_hat, smooth, a_seq, a_mu, a_var, a_mu_pred_seq, x_gene = [], [], [], [], [], [], []
    for i in range(num_batches):
        slc = slice(i * model.config.batch_size, (i + 1) * model.config.batch_size)
        feed_dict = {model.x: model.test_data.images[slc],
                     model.kf.u: model.test_data.controls[slc],
                     model.ph_steps: model.test_data.timesteps,
                     model.mask: mask_test}
        x_hat_, smooth_, a_seq_, a_mu_, a_var_, a_mu_pred_seq_ = sess.run(
            [
                model.model_vars["x_hat"],         # x <- a <- x
                model.model_vars["smooth"][0],     # z <- a <- x
                model.model_vars["a_seq"],         # a <- x (sequential)
                model.model_vars["a_mu"],          # a <- x (squeezed)
                model.model_vars["a_var"],         # a <- x (var)
                model.model_vars["a_mu_pred_seq"]  # a <- z <- a <- x
            ],
            feed_dict=feed_dict)
        x_gene_ = sess.run(model.model_vars["x_hat"], feed_dict={model.model_vars["a_seq"]: a_mu_pred_seq_,
                                                               model.ph_steps: model.n_steps_gen})
        x_hat.append(x_hat_)
        smooth.append(smooth_)
        a_seq.append(a_seq_)
        a_mu.append(a_mu_)
        a_var.append(a_var_)
        a_mu_pred_seq.append(a_mu_pred_seq_)
        x_gene.append(x_gene_)
    x_hat_seq = np.vstack(x_hat)
    smooth = np.vstack(smooth)
    a_seq = np.vstack(a_seq)
    a_mu = np.vstack(a_mu)
    a_var = np.vstack(a_var)
    a_mu_pred_seq = np.vstack(a_mu_pred_seq)
    x_gene_seq = np.vstack(x_gene)
    np.savez_compressed(
        "outputs/2dspring_{}_{}dim".format(pattern, n_lat),
        x_reconst=x_hat_seq, s_smooth=smooth, z_m=a_seq, z_smooth=a_mu_pred_seq, x_smooth=x_gene_seq
    )